In [3]:
import pandas as pd
import re

In [4]:
df = pd.read_csv("food_coded.csv")

In [5]:
df.head()

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,soup,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight
0,2.4,2,1,430,NaN,315.0,1,none,we dont have comfort,9.0,...,1.0,1.0,1,1165.0,345,car racing,5,1,1315,187
1,3.654,1,1,610,3.0,420.0,2,"chocolate, chips, ice cream","Stress, bored, anger",1.0,...,1.0,1.0,2,725.0,690,Basketball,4,2,900,155
2,3.3,1,1,720,4.0,420.0,2,"frozen yogurt, pizza, fast food","stress, sadness",1.0,...,1.0,2.0,5,1165.0,500,none,5,1,900,I'm not answering this.
3,3.2,1,1,430,3.0,420.0,2,"Pizza, Mac and cheese, ice cream",Boredom,2.0,...,1.0,2.0,5,725.0,690,NaN,3,1,1315,"Not sure, 240"
4,3.5,1,1,720,2.0,420.0,2,"Ice cream, chocolate, chips","Stress, boredom, cravings",1.0,...,1.0,1.0,4,940.0,500,Softball,4,2,760,190


In [6]:
df = df.iloc[:, [52, 41, 22, 20, 15, 2, 1, 7, 33, 60]]

In [7]:
df.head()

,sports,marital_status,exercise,employment,drink,breakfast,Gender,comfort_food,healthy_feeling,weight
0,1.0,1.0,1.0,3.0,1.0,1,2,none,2,187
1,1.0,2.0,1.0,2.0,2.0,1,1,"chocolate, chips, ice cream",5,155
2,2.0,2.0,2.0,3.0,1.0,1,1,"frozen yogurt, pizza, fast food",6,I'm not answering this.
3,2.0,2.0,3.0,3.0,2.0,1,1,"Pizza, Mac and cheese, ice cream",7,"Not sure, 240"
4,1.0,1.0,1.0,2.0,2.0,1,1,"Ice cream, chocolate, chips",6,190


In [8]:
df.loc[:, "is_active"] = df.sports == 1
df.loc[:, "is_single"] = df.marital_status == 1
df.loc[:, "is_sporty"] = df.exercise == 1
df.loc[:, "is_employed"] = (df.employment == 1) | (df.employment == 2)
df.loc[:, "is_healthy_eating"] = (df.drink == 1) & (df.breakfast == 1)
df.loc[:, "gender"] = df.Gender - 1
df.loc[:, "weight"] = df.weight.fillna(0).apply(lambda x: int(int(*re.findall("[0-9]+", str(x))) / 2.2))

In [9]:
cols = ["is_sporty", "is_single", "is_active", "is_employed", "gender", "is_healthy_eating", "weight", "comfort_food", "healthy_feeling"]


In [10]:
df = df[cols]

In [11]:
categories = set()

In [12]:
for _, row in df.comfort_food.iteritems():
    try:
        categories |= {a.lower().replace("and", "").strip() for a in row.split(",")}
    except:
        pass

In [13]:
veg = ["broccoli", "carrots", "cucumber", "fruit", "grapes", "watermelon", "rice"]
sport = ["almonds", "cereal", "yogurt", "garlic", "omelet", "shake"]
ten_min = ["burger", "wings", "chips", "popcorn", "nugg", "cookie",
          "donut", "fast", "snacks"]
classic = ["beef", "grilled", "lasagn", "potato", "meat", "noodle", "peanut", "pasta", "soup", "toast",
          "burritos", "micro", "chocolate"]
premium = ["curry", "chinese", "indian", "korean", "pizza", "spaghetti", "sushi", "wine", "truffles"]
holiday = ["ice", "cake", "brownies", "macaro", "mozzarella"]

In [14]:
for _, row in df.comfort_food.iteritems():
    try:
        user_input = [a.lower().replace("and", "").strip() for a in row.split(",")]
        result = [0 for _ in range(6)]
        for i, cat_gr in enumerate([veg, sport, ten_min, classic, premium, holiday]):
            for x in user_input:
                for cat in cat_gr:
                    if cat in x:
                        result[i] = 1
        for j in range(len(result)):
            df.loc[_, f'target_{j}'] = result[j]
    except:
        for i in range(6):
            df.loc[i, f'target_{i}'] = 0

In [15]:
X = df.loc[:, cols[:7]].astype(int)
y = df.loc[:, [f'target_{i}' for i in range(6)]].fillna(0).astype(int)

In [16]:
y.sum()

target_0    13
target_1     8
target_2    65
target_3    60
target_4    50
target_5    66
dtype: int64

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

сильные фичи
* 52 - спорт да(1) , нет (2, 99)
* 41 - отношения один(1), остальное(2,3,4,5,6)
* 22 - спорт да (1,2), нет (3,4,5)

слабые фичи
* 20 - работа занят(1,2), нет(3,4)
* 15 - здоровое питание да(1), нет(2)
* 3 - здоровый завтрак да(1), нет(2)
* 2 - пол ж(1), м(2)

таргет фичи
* 8 - комфортная еда
* 33 - ощущения здоровья от 1 очень здоров до 10 нездоров

In [603]:
import lightgbm as lgb

In [617]:
# other scikit-learn modules
estimator = OneVsRestClassifier(lgb.LGBMClassifier(num_leaves=31))

param_grid = {
    'estimator__max_depth': [1, 2, 3],
    'estimator__subsample': np.arange(0.6, 1.2, 0.2),
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__n_estimators': [10, 20, 40, 60]
}

gbm = GridSearchCV(estimator, param_grid)

gbm.fit(X_train, y_train)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'estimator__learning_rate': 0.1, 'estimator__max_depth': 1, 'estimator__n_estimators': 20, 'estimator__subsample': 0.8}


In [622]:
gbm.best_estimator_

OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=20,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.8,
        subsample_for_bin=200000, subsample_freq=1),
          n_jobs=1)

In [623]:
from sklearn.externals import joblib
joblib.dump(gbm.best_estimator_, 'gbm.pkl') 

['gbm.pkl']

In [20]:
gbm = joblib.load("gbm.pkl")

In [26]:
gbm.estimator.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_depth': 1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 20,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 0.8,
 'subsample_for_bin': 200000,
 'subsample_freq': 1}